## Libraries und Vairablen

In [53]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by  import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import re

In [54]:
buchhaus_new_last_30 = 'https://www.buchhaus.ch/de/heute/last30'

## Webscraping

In [55]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)
#load the page
driver.get(buchhaus_new_last_30)

# data frame
buch_df = pd.DataFrame(columns=['Titel',  
                            'Preis',
                            'Autor', 
                            'details', 
                            'Genre',
                            'Text'])

for page in range(1, 11):
    print('Scraping books from page {}'.format(page))

    # scrape all hyperlinks
    links = driver.find_elements(By.XPATH, '/html/body/div[2]/div/div/main/div[3]/div[2]/div[4]/div//a')

    # get the href attribute
    links = [link.get_attribute('href') for link in links]

    # remove all links that are not books
    links = [link for link in links if 'buecher' in link]

    for book in links:

        try:
            driver.get(book)

            # get the data simple data
            try: titel = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[1]/div[2]/div[1]/h1/span/span').text
            except: titel = 'nan'

            try: preis = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[1]/div[2]/div[2]/div/div/span[2]/span').text
            except: preis = 'nan'

            try: autor = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[1]/div[2]/div[1]/div[2]/div').text
            except: autor = 'nan'

            # get the 'Buchbeschreibungen'
            try: text = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[3]/div[1]/div[2]/div/span/span/span').text
            except: text = 'nan'

            # get the genre
            parts = book.split('/')
            #find the index of the genre
            start_index = parts.index('buecher')
            end_index = parts.index('detail')
            # get the genres
            genre = parts[start_index+1:end_index]
            genre = ' '.join(genre)

            # get the 'Buchdetails'
            try: details = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[3]/div[2]/div[2]/div').text
            except: details = 'nan'

        # Fehlermeldung
        except: 
            print('error mit Buch {} auf Seite {}'.format(book, page))
            continue
    

        df_temp = pd.DataFrame({'Titel': titel,  
                        'Preis': preis,
                        'Autor': autor,
                        'details': details,
                        'Genre' : genre,
                        'Text': text
                        }, index=[0])
        buch_df = pd.concat([buch_df, df_temp], ignore_index=True)
        
        #back to the main page
        driver.get(buchhaus_new_last_30)

    # go to the next page
    if page == 1: driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[3]/div[2]/div[1]/div[5]/div/a').click()
    else: driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[3]/div[2]/div[1]/div[5]/div/a[2]').click()
    time.sleep(1)

driver.close()

C:\Users\yanni\AppData\Local\Temp\ipykernel_16180\2240200850.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


Scraping books from page 1
Scraping books from page 1
Scraping books from page 1
Scraping books from page 1
Scraping books from page 1
Scraping books from page 1
Scraping books from page 1
Scraping books from page 1
Scraping books from page 1
Scraping books from page 1
Scraping books from page 1
Scraping books from page 1
Scraping books from page 1
Scraping books from page 1
Scraping books from page 1
Scraping books from page 1
Scraping books from page 1
Scraping books from page 1
Scraping books from page 1
Scraping books from page 1
Scraping books from page 1
Scraping books from page 1
Scraping books from page 1
Scraping books from page 1
Scraping books from page 1
Scraping books from page 1
Scraping books from page 1
Scraping books from page 1
Scraping books from page 1
Scraping books from page 2
Scraping books from page 2
Scraping books from page 2
Scraping books from page 2
Scraping books from page 2
Scraping books from page 2
Scraping books from page 2
Scraping books from page 2
S

In [56]:
links

['https://www.buchhaus.ch/de/buecher/belletristik/romane/detail/ISBN-9783442316458/Lundberg-Sofia/Wo-wir-uns-trafen',
 'https://www.buchhaus.ch/de/buecher/belletristik/romane/detail/ISBN-9783442316458/Lundberg-Sofia/Wo-wir-uns-trafen',
 'https://www.buchhaus.ch/de/buecher/belletristik/krimi/detail/ISBN-9783257071924/McCarten-Anthony/Going-Zero',
 'https://www.buchhaus.ch/de/buecher/belletristik/krimi/detail/ISBN-9783257071924/McCarten-Anthony/Going-Zero',
 'https://www.buchhaus.ch/de/buecher/taschenbuch/romane/detail/ISBN-9783442493968/Swan-Karen/Sommern%C3%A4chte-in-Paris',
 'https://www.buchhaus.ch/de/buecher/belletristik/krimi/detail/ISBN-9783423263597/Clarke-Lucy/One-of-the-Girls',
 'https://www.buchhaus.ch/de/buecher/belletristik/romane/detail/ISBN-9783458643692/Yagisawa-Satoshi/Die-Tage-in-der-Buchhandlung-Morisaki',
 'https://www.buchhaus.ch/de/buecher/taschenbuch/biografien/detail/ISBN-9783499006340/Arnim-Gabriele-von/Das-Leben-ist-ein-vor%C3%BCbergehender-Zustand',
 'https://w

## Datawrangling

In [57]:
buch_df.shape

(52, 6)

In [58]:
'''
short algorithm description:
1. List all possible keys: These are the categories that precede the actual values in the data string.
2. Create a regular expression: This regular expression is designed to match each key, followed by any characters until the next key is found or until the end of the string.
3. Find all matches in the data: Use the 're.findall' function to apply the regular expression to the data string. 
This returns a list of tuples containing the key and the associated value for each match.
4. Convert the matches into a Dictionnary in the Dataframe.
'''

def extract_details(details):
    # Key words which can be find in the string
    keys = ["ISBN/GTIN", "Produktart", "Einbandart", "Verlag", "Erscheinungsdatum", "Auflage", "Reihe", "Reihen-Nr.", "Seiten", "Sprache", "Masse", "Artikel-Nr."]

    # generate a regex which can find all the keys in the string
    regex = "(" + "|".join(keys) + ")(.*?)(?=" + "|".join(keys) + "|$)"

    # finds all key value pairs in the string
    matches = re.findall(regex, details)
    
    # returns a dictionary with key value pairs
    return dict((k.strip(), v.strip()) for k, v in matches)

In [59]:
def data_wrangling_pipeline(df):
    # apply the function to the data frame (create a new column with extracted details)
    df['details_dict'] = df['details'].apply(extract_details)
    # create new dataframe with the details
    details_df = df['details_dict'].apply(pd.Series, dtype= 'object')
    # concat both dataframes
    new_df = pd.concat([buch_df, details_df], axis=1)
    # drop the old details columns
    new_df.drop(['details', 'details_dict'], axis=1, inplace=True)
    return new_df

In [60]:
buch_df = data_wrangling_pipeline(buch_df)

#drop all rows where the title is nan
buch_df = buch_df[buch_df['Titel'] != 'nan']

#drop duplicates
buch_df.drop_duplicates(subset=['Titel'], inplace=True)

In [61]:
buch_df

,Titel,Preis,Autor,Genre,Text,ISBN/GTIN,Produktart,Einbandart,Verlag,Erscheinungsdatum,Auflage,Reihe,Seiten,Sprache,Masse,Artikel-Nr.
0,Atlas - Die Geschichte von Pa Salt,32.00,"Riley, LucindaWhittaker, HarryHauser, SonjaÜbe...",belletristik romane,"Paris, 1928. Ein Junge wird gerade noch rechtz...",978-3-442-31567-3,Buch,Gebunden,Goldmann,11.05.2023Erstverkaufstag11.05.2023,Deutsche Erstausgabe,n-Nr.08,,NaN,NaN,NaN
1,Atlas - Die Geschichte von Pa Salt,32.00,"Riley, LucindaWhittaker, HarryHauser, SonjaÜbe...",belletristik romane,"Paris, 1928. Ein Junge wird gerade noch rechtz...",978-3-442-31567-3,Buch,Gebunden,Goldmann,11.05.2023Erstverkaufstag11.05.2023,Deutsche Erstausgabe,n-Nr.08,,NaN,NaN,NaN
2,Der Feind,24.90,"Brand, Christine",belletristik krimi,Ein bizarre Mordserie an Männern sowie Schüsse...,978-3-7645-0771-8,Buch,Paperback,Blanvalet,26.04.2023,1. A.,n-Nr.05,,Deutsch,NaN,NaN
3,Der Feind,24.90,"Brand, Christine",belletristik krimi,Ein bizarre Mordserie an Männern sowie Schüsse...,978-3-7645-0771-8,Buch,Paperback,Blanvalet,26.04.2023,1. A.,n-Nr.05,,Deutsch,NaN,NaN
4,Die Krume Brot,29.00,nan,belletristik schweizer,"Adelina, Tochter italienischer Einwanderer, ar...",978-3-498-00320-3,Buch,Gebunden,Rowohlt,18.04.2023,,NaN,,Deutsch,"Breite 133 mm, Höhe 209 mm, Dicke 23 mmGewicht...",NaN
5,Die Krume Brot,29.00,nan,belletristik schweizer,"Adelina, Tochter italienischer Einwanderer, ar...",978-3-498-00320-3,Buch,Gebunden,Rowohlt,18.04.2023,,NaN,,Deutsch,"Breite 133 mm, Höhe 209 mm, Dicke 23 mmGewicht...",NaN
6,Anuschka und Finn,20.00,"Schawinski, Roger",fachbuecher geschichte schweiz,"Roger Schawinski über einen Medienskandal, der...",978-3-033-09890-9,Buch,Paperback,Radio 1 AG,12.05.2023,NaN,NaN,,Deutsch,"Breite 144 mm, Höhe 221 mm, Dicke 15 mmGewicht...",58408389
7,Elternabend,23.90,"Fitzek, Sebastian",belletristik romane,Stell dir vor ...\n... du musst eine halbe Ewi...,978-3-426-28413-1,Buch,Paperback,Droemer/Knaur,26.04.2023Erstverkaufstag26.04.2023,,NaN,,Deutsch,"Breite 135 mm, Höhe 210 mm, Dicke 25 mm",NaN
8,Troubadour,35.00,"Walker, MartinWindgassen, MichaelÜbersetzung",belletristik krimi,Bruno steckt mitten in den Vorbereitungen für ...,978-3-257-07237-2,Buch,Gebunden,Diogenes,26.04.2023,1. A.,n-Nr.15,,Deutsch,NaN,NaN
9,Der Magier im Kreml,36.90,"Da Empoli, GiulianoMeßner, MichaelaÜbersetzung",belletristik romane,SPIEGEL-BESTSELLER UND VIELFACH AUSGEZEICHNET\...,978-3-406-79993-8,Buch,Gebunden,Beck,24.04.2023,NaN,NaN,,Deutsch,"Breite 139 mm, Höhe 217 mm, Dicke 25 mmGewicht...",55652787


## Export

In [62]:
# save as xlsx
buch_df.to_excel('../data/excelfiles/df_neuheiten_blatter.xlsx', index=False)
# save as feather
buch_df.to_feather('../data/feather/df_neuheiten_blatter.feather')

## Ausblick

Ausblick:

* Problematik mit 5 Bücher scrapen muss behoben werden
* Bücher in den nächsten 30 Tagen könnten auch gescrapet werden